<a href="https://colab.research.google.com/github/tomheston/fragility-metrics/blob/main/notebooks/contingency_rxc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
# Complete Evidence Framework: r×c Contingency Tables (Independent Samples)
# v.25-NOV-2025
# Multinomial outcome analysis with complete p-fr-nb triplet
#
# Input:
#   r×c contingency table (r rows, c columns)
#   All cells as counts (non-negative integers)
#
# Output:
#   p  = p-value from chi-square test (or Fisher's exact for small N)
#   fr = GFQ (Global Fragility Quotient, stability of classification)
#   nb = RQ (Risk Quotient, distance from independence)
#
# GFQ Formula:
#   GFI = minimum number of cell-to-cell reallocations to flip significance
#   GFQ = GFI / N  (normalized [0,1])
#   Uses path-independent BFS with adaptive stepping
#
# RQ Formula:
#   RRI = (1/k) Σ|O - E|  (raw distance from independence)
#   RQ = RRI / (N/k)  (normalized [0,1])
#   where k = (r-1)(c-1) = independent cells
#         O = observed counts, E = expected under independence
#
# Test Selection:
#   - Chi-square: N > 1000 OR all expected counts ≥ 10
#   - Fisher's exact: Otherwise (2×2 only)
#   - For r×c with small cells: Chi-square with warning
#
# Interpretation:
#   fr ∈ [0,1]: 0 = fragile, 1 = stable
#   nb ∈ [0,1]: 0 = independent, 1 = maximally associated
#
# Limits:
#   GFI computation: N ≤ 50000 (use RQ/RI for larger samples)
#   Adaptive stepping for efficiency with large N
#
# IF YOU USE THIS CALCULATOR PLEASE CITE:
# Heston, T. F. (2025). Fragility Metrics Toolkit [Software]. Zenodo.
#   https://doi.org/10.5281/zenodo.17254763
#
# © Thomas F. Heston 2025. CC-BY 4.0

# ----- SciPy availability guard -----
try:
    import scipy
except ImportError:
    try:
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "scipy"])
        import scipy
    except Exception:
        print("Please install scipy: pip install scipy")
        raise

import numpy as np
from scipy.stats import chi2_contingency, fisher_exact
from collections import deque
import sys

# ========== CONFIGURABLE PARAMETERS ==========
ALPHA = 0.05
N_THRESHOLD = 1000         # Sample size threshold for chi-square vs Fisher
MIN_CELL_THRESHOLD = 10    # Minimum expected count for chi-square validity
GFI_THRESHOLD = 50000      # Don't compute GFI for N > this
GFI_COARSE_STEP = None     # None = adaptive (recommended)
GFI_MAX_DEPTH = None       # None = adaptive (recommended)
# =============================================

# ---------- Core Utilities ----------

def test_p(table, use_chi2=False):
    """
    Unified significance test for r×c tables.

    Parameters:
    -----------
    table : 2D array-like
        Contingency table
    use_chi2 : bool
        If True, use chi-square; if False, use Fisher (2×2 only)

    Returns:
    --------
    float : p-value
    """
    table = np.array(table)
    if use_chi2 or table.shape[0] > 2 or table.shape[1] > 2:
        _, p, _, _ = chi2_contingency(table)
        return p
    else:
        # Fisher's exact for 2×2
        _, p = fisher_exact(table, alternative="two-sided")
        return p


def is_significant(p, alpha=ALPHA):
    """Check if p-value indicates significance."""
    return p <= alpha


# ---------- RRI / RQ (Robustness) ----------

def compute_rq(table):
    """
    Calculate Risk Quotient (RQ) for r×c table.

    RRI = (1/k) Σ|O - E|
    RQ = RRI / (N/k)

    where k = (r-1)(c-1) = independent cells

    Parameters:
    -----------
    table : 2D array-like
        Contingency table

    Returns:
    --------
    dict : {'RRI': float, 'RQ': float, 'k': int}

    Reference:
    ----------
    FRAGILITY_METRICS v9.6, §4.1
    """
    table = np.array(table, dtype=float)
    r, c = table.shape
    N = table.sum()

    if N == 0:
        return {'RRI': None, 'RQ': None, 'k': None}

    # Expected counts under independence
    row_totals = table.sum(axis=1, keepdims=True)
    col_totals = table.sum(axis=0, keepdims=True)
    expected = (row_totals @ col_totals) / N

    # Number of independent cells
    k = (r - 1) * (c - 1)

    # RRI and RQ
    RRI = np.abs(table - expected).sum() / k if k > 0 else 0
    RQ = RRI / (N / k) if k > 0 else 0

    return {
        'RRI': float(RRI),
        'RQ': float(RQ),
        'k': int(k)
    }


# ---------- GFI / GFQ (Fragility) ----------

def compute_gfi_gfq(table, alpha=ALPHA, store_path=False):
    """
    Calculate Global Fragility Index (GFI) and Quotient (GFQ) for r×c table.
    Uses progressive multi-resolution adaptive step search.

    Parameters:
    -----------
    table : 2D array-like
        Contingency table
    alpha : float
        Significance level
    store_path : bool
        If True, store witness path (memory intensive)

    Returns:
    --------
    dict : Contains GFI, GFQ, baseline_p, final_p, test_used

    Reference:
    ----------
    FRAGILITY_METRICS v9.6, §3.3
    """
    table = np.array(table, dtype=int)
    r, c = table.shape

    # Validate
    if np.any(table < 0):
        raise ValueError("All cells must be non-negative")

    N = table.sum()

    if N == 0:
        return {
            "GFI": None, "GFQ": None,
            "baseline_p": None, "baseline_state": None,
            "final_p": None, "witness_path": [],
            "test_used": None,
            "note": "Empty table."
        }

    if N > GFI_THRESHOLD:
        use_chi2 = True
        return {
            "GFI": None, "GFQ": None,
            "baseline_p": test_p(table, use_chi2),
            "baseline_state": None,
            "final_p": None, "witness_path": [],
            "test_used": "chi2",
            "note": f"GFI not computed for N > {GFI_THRESHOLD}. Use RQ instead."
        }

    # Determine test
    use_chi2 = (N > N_THRESHOLD) or np.all(compute_expected(table) >= MIN_CELL_THRESHOLD)

    # Adaptive parameters
    if GFI_COARSE_STEP is not None:
        initial_coarse_step = GFI_COARSE_STEP
    else:
        if N < 1000:
            initial_coarse_step = 1
        elif N < 1500:
            initial_coarse_step = 2
        elif N < 2500:
            initial_coarse_step = 3
        else:
            initial_coarse_step = 5

    if GFI_MAX_DEPTH is not None:
        max_depth = GFI_MAX_DEPTH
    else:
        max_depth = 50 + N // 2
        max_depth = min(max_depth, 1000)

    def get_adaptive_step(p_now, alpha, initial_step):
        """Progressive step size based on distance from alpha."""
        if not np.isfinite(p_now) or p_now <= 0:
            return initial_step

        ratio = p_now / alpha if p_now < alpha else alpha / p_now

        if ratio > 0.7:
            return 1
        elif ratio > 0.5:
            return min(2, initial_step)
        elif ratio > 0.2:
            return min(3, initial_step)
        elif ratio > 0.1:
            return min(4, initial_step)
        else:
            return initial_step

    base_p = test_p(table, use_chi2)
    base_state = is_significant(base_p, alpha)

    def table_to_state(tbl):
        """Convert table to state tuple (flattened, excluding last cell)."""
        flat = tbl.flatten()
        return tuple(flat[:-1])

    def state_to_table(state, N, shape):
        """Convert state back to table."""
        flat = list(state) + [N - sum(state)]
        return np.array(flat).reshape(shape)

    def neighbors_adaptive(state, N, shape, step_size):
        """Generate neighbors with specified step size."""
        tbl = state_to_table(state, N, shape)
        flat = tbl.flatten()
        n_cells = len(flat)

        for src in range(n_cells):
            if flat[src] < step_size:
                continue
            for dst in range(n_cells):
                if dst == src:
                    continue
                new_flat = flat.copy()
                new_flat[src] -= step_size
                new_flat[dst] += step_size
                new_state = tuple(new_flat[:-1])
                yield new_state, (src, dst, step_size)

    start = table_to_state(table)
    visited = {start: 0}
    q = deque()

    initial_step = get_adaptive_step(base_p, alpha, initial_coarse_step)
    q.append((start, 0, [] if store_path else None, initial_step))

    max_moves_explored = 0

    while q:
        state, moves, path, step = q.popleft()

        if moves > max_depth:
            max_moves_explored = max(max_moves_explored, moves)
            continue

        # Check current state
        tbl = state_to_table(state, N, table.shape)
        p_now = test_p(tbl, use_chi2)

        # Found flip?
        if is_significant(p_now, alpha) != base_state:
            return {
                "GFI": moves,
                "GFQ": moves / N if N else None,
                "baseline_p": base_p,
                "baseline_state": ("significant" if base_state else "non-significant"),
                "final_p": p_now,
                "witness_path": path if store_path else [],
                "test_used": "chi2" if use_chi2 else "fisher_exact"
            }

        # Determine step size for next moves
        next_step = get_adaptive_step(p_now, alpha, initial_coarse_step)

        # Generate neighbors
        for nxt, move in neighbors_adaptive(state, N, table.shape, next_step):
            new_moves = moves + move[2]

            if nxt not in visited or new_moves < visited[nxt]:
                visited[nxt] = new_moves
                new_path = (path + [move]) if store_path else None
                q.append((nxt, new_moves, new_path, next_step))

        max_moves_explored = max(max_moves_explored, moves)

    return {
        "GFI": None,
        "GFQ": None,
        "baseline_p": base_p,
        "baseline_state": ("significant" if base_state else "non-significant"),
        "final_p": base_p,
        "witness_path": [],
        "test_used": "chi2" if use_chi2 else "fisher_exact",
        "note": f"GFI > {max_moves_explored} (search limited to {max_depth} moves)"
    }


def compute_expected(table):
    """Calculate expected counts under independence."""
    table = np.array(table, dtype=float)
    N = table.sum()
    if N == 0:
        return table
    row_totals = table.sum(axis=1, keepdims=True)
    col_totals = table.sum(axis=0, keepdims=True)
    return (row_totals @ col_totals) / N


# ---------- Complete Evidence Calculator ----------

def contingency_table_complete(table, alpha=ALPHA):
    """
    Calculate complete p-fr-nb triplet for r×c contingency table.

    Parameters:
    -----------
    table : 2D array-like
        Contingency table
    alpha : float
        Significance level

    Returns:
    --------
    dict : Complete evidence assessment
    """
    table = np.array(table, dtype=int)
    r, c = table.shape
    N = table.sum()

    # Determine test
    expected = compute_expected(table)
    use_chi2 = (N > N_THRESHOLD) or np.all(expected >= MIN_CELL_THRESHOLD)

    # p-value
    p_value = test_p(table, use_chi2)

    # fr (GFQ)
    if N <= GFI_THRESHOLD:
        gfi_result = compute_gfi_gfq(table, alpha=alpha, store_path=False)
        gfi = gfi_result.get("GFI")
        gfq = gfi_result.get("GFQ")
        gfi_note = gfi_result.get("note", "")
    else:
        gfi = None
        gfq = None
        gfi_note = f"GFI not computed for N > {GFI_THRESHOLD}"

    # nb (RQ)
    rq_result = compute_rq(table)

    return {
        'p': p_value,
        'fr': gfq,
        'nb': rq_result['RQ'],
        'GFI': gfi,
        'RRI': rq_result['RRI'],
        'k_independent': rq_result['k'],
        'table_shape': (r, c),
        'N_total': N,
        'test_used': "chi2" if use_chi2 else "fisher_exact",
        'expected_counts': expected,
        'min_expected': np.min(expected),
        'gfi_note': gfi_note
    }


# ---------- Interpretation Functions ----------

def interpret_fragility(fr):
    """Interpret GFQ (fragility quotient)."""
    if fr is None:
        return "not computed"
    if fr < 0.01:
        return "extremely fragile"
    elif fr < 0.05:
        return "very fragile"
    elif fr < 0.10:
        return "fragile"
    elif fr < 0.25:
        return "mildly stable"
    elif fr < 0.40:
        return "moderate stability"
    else:
        return "very stable"


def interpret_robustness(nb):
    """Interpret RQ (robustness)."""
    if nb is None:
        return "not computed"
    if nb < 0.05:
        return "at independence (no association)"
    elif nb < 0.10:
        return "near independence (minimal association)"
    elif nb < 0.25:
        return "moderate distance from independence"
    elif nb < 0.50:
        return "clear separation from independence"
    else:
        return "far from independence (strong association)"


# ---------- Summary Interface ----------

def contingency_summary(result):
    """Display complete p-fr-nb evidence assessment."""
    def fmt_f(x):
        return "NA" if x is None or not np.isfinite(x) else f"{x:.6f}"

    def fmt_i(x):
        return "NA" if x is None else str(int(x))

    r, c = result['table_shape']
    print(f"Table dimensions: {r} × {c}")
    print(f"N_total = {fmt_i(result['N_total'])}")
    print(f"Test used: {result['test_used']}")
    print(f"Min expected count: {fmt_f(result['min_expected'])}")

    if result['min_expected'] < 5:
        print("⚠ Warning: Some expected counts < 5 (chi-square may be unreliable)")
    print()

    # Complete evidence triplet
    print("=" * 50)
    print("COMPLETE EVIDENCE ASSESSMENT (p-fr-nb)")
    print("=" * 50)
    print(f"p = {fmt_f(result['p'])}")
    print(f"fr (GFQ) = {fmt_f(result['fr'])}")
    print(f"nb (RQ) = {fmt_f(result['nb'])}")
    print("=" * 50)
    print()

    # Interpretations
    interp_fr = interpret_fragility(result['fr'])
    interp_nb = interpret_robustness(result['nb'])

    print("INTERPRETATION:")
    print(f"Fragility: {interp_fr}")
    print(f"Robustness: {interp_nb}")
    print()

    # Additional metrics
    print(f"GFI (raw count) = {fmt_i(result['GFI'])}")
    print(f"RRI (raw distance) = {fmt_f(result['RRI'])}")
    print(f"Independent cells (k) = {fmt_i(result['k_independent'])}")

    if result['gfi_note']:
        print(f"Note: {result['gfi_note']}")

    # Significance classification
    is_sig = result['p'] <= ALPHA
    print()
    print(f"Classification: {'SIGNIFICANT' if is_sig else 'NON-SIGNIFICANT'} at α={ALPHA}")


# ---------- CLI Entry Point ----------

def main():
    print("Complete Evidence Framework: r×c Contingency Tables")
    print("=" * 50)
    print("Calculate p-fr-nb triplet for multinomial outcomes")
    print()

    # Get table dimensions
    r = int(input("Number of rows (r): ").strip())
    c = int(input("Number of columns (c): ").strip())

    if r < 2 or c < 2:
        print("Error: Need at least 2×2 table")
        sys.exit(1)

    print()
    print(f"Enter all {r*c} cell counts (row by row):")
    print()

    # Get cell values
    table = []
    for i in range(r):
        row = []
        for j in range(c):
            val = int(input(f"  Cell ({i+1},{j+1}): ").strip())
            if val < 0:
                print("Error: Cell counts must be non-negative")
                sys.exit(1)
            row.append(val)
        table.append(row)

    print()
    print("Input table:")
    table_array = np.array(table)
    print(table_array)
    print()

    # Calculate complete evidence
    result = contingency_table_complete(table, alpha=ALPHA)
    contingency_summary(result)


if __name__ == "__main__":
    main()


== BW Ticker Drawdown Check ==

+--------+---------+----------+------------+--------+
| Ticker | Current | 12W High | Drawdown % | Status |
+--------+---------+----------+------------+--------+
|  BITX  |  52.12  |  60.65   |   14.06%   |   OK   |
|  MSTX  |  35.69  |  47.23   |   24.43%   |   OK   |
|  PLTR  | 127.72  |  133.17  |   4.09%    |   OK   |
|  SOXL  |  19.18  |  20.94   |   8.38%    |   OK   |
|  TQQQ  |  74.21  |  74.21   |   0.00%    |   OK   |
+--------+---------+----------+------------+--------+

== Closest OTM ==

+------------+--------+--------+--------+---------+-----------+--------------+-----+---------------+
| Expiration | Ticker |  Spot  | Strike | Premium | CashYield | AssignedGain | DTE | MeetsCriteria |
+------------+--------+--------+--------+---------+-----------+--------------+-----+---------------+
| 2025-06-13 |  BITX  | 52.12  |  53.0  |  1.59   |   3.01%   |    4.75%     |  5  |      Yes      |
| 2025-06-13 |  MSTX  | 35.69  |  36.0  |  1.57   |   4.3